In [5]:
# =========================
# Standard library
# =========================
import os
import time
import copy
import warnings

warnings.filterwarnings("ignore")

# =========================
# Core scientific stack
# =========================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

# =========================
# PyTorch + TorchVision
# =========================
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
from torchvision import transforms as T
from torchvision import models as tv_models

# =========================
# timm
# =========================
import timm
from timm.loss import LabelSmoothingCrossEntropy

# =========================
# Metrics (scikit-learn)
# =========================
import sklearn.metrics as sk_metrics
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    cohen_kappa_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    classification_report,
    roc_curve,
    auc,
)


In [6]:
# Assuming y_true and y_pred are already defined
def get_classes(data_dir):
    """Get class names from dataset directory."""
    all_data = datasets.ImageFolder(data_dir)
    return all_data.classes

def get_data_loaders(data_dir, batch_size, train=False):
    """Prepare data loaders for training, validation, and testing."""
    if train:
        # Data augmentation for training
        transform = T.Compose([
            T.RandomHorizontalFlip(),
            T.RandomVerticalFlip(),
          
            T.Resize((299, 299)),
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        train_data = datasets.ImageFolder(os.path.join(data_dir, "train/"), transform=transform)
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
        return train_loader, len(train_data)
    else:
        # Validation and test data transformations
        transform = T.Compose([
            T.Resize((299, 299)),
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        val_data = datasets.ImageFolder(os.path.join(data_dir, "validation/"), transform=transform)
        test_data = datasets.ImageFolder(os.path.join(data_dir, "test/"), transform=transform)
        val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=4)
        test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4)
        return val_loader, test_loader, len(val_data), len(test_data)

# Dataset paths and loaders
dataset_path = "new_directories"
batch_size = 32  # Update batch size for efficiency

(train_loader, train_data_len) = get_data_loaders(dataset_path, batch_size, train=True)
(val_loader, test_loader, valid_data_len, test_data_len) = get_data_loaders(dataset_path, batch_size, train=False)

classes = get_classes("new_directories/train")
num_classes = len(classes)

dataloaders = {"train": train_loader, "val": val_loader}
dataset_sizes = {"train": train_data_len, "val": valid_data_len}

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the pre-trained Inception-v3 model
model = models.inception_v3(pretrained=True)

# Update the fully connected layer for the number of classes
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Send the model to the device
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# Learning rate scheduler
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.97)






NameError: name 'models' is not defined

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=20):
    """Train and validate the model."""
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluation mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)

                    # Handle auxiliary logits for Inception-v3
                    if isinstance(outputs, tuple) or hasattr(outputs, "logits"):
                        main_outputs = outputs.logits if hasattr(outputs, "logits") else outputs[0]
                        aux_outputs = outputs.aux_logits if hasattr(outputs, "aux_logits") else None
                        loss = criterion(main_outputs, labels)
                        if aux_outputs is not None:  # Add auxiliary loss if present
                            loss += 0.4 * criterion(aux_outputs, labels)
                    else:
                        main_outputs = outputs
                        loss = criterion(main_outputs, labels)

                    _, preds = torch.max(main_outputs, 1)  # Extract predictions from logits

                    # Backward pass and optimization in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            # Step the scheduler in the training phase
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

    return model

# Train the model
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=20)

In [ ]:


y_true = []
y_proba = []
y_pred = []

# Disable gradient calculation
with torch.no_grad():
    for x, y in tqdm(test_loader):
        # Forward pass
        output = model(x.to(device))
        
        # Store true labels
        y_true.extend(y.numpy())
        
        # Get predicted probabilities (softmax output)
        probabilities = torch.softmax(output, dim=1)  # Assuming it's a multi-class classification
        y_proba.extend(probabilities.cpu().numpy())  # Store probabilities
        
        # Get predicted classes (argmax for final class prediction)
        pred = torch.argmax(probabilities, axis=1).cpu().numpy()
        y_pred.extend(pred)





In [ ]:

cm = confusion_matrix(y_true, y_pred)

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)

# Plot the confusion matrix without percentages
disp.plot(cmap="Blues", values_format='')
plt.title("Inception-v3", fontsize=16)
plt.show()


In [ ]:
print(classification_report(y_true, y_pred, target_names=classes, digits=4))

In [ ]:

finaltrain = pd.DataFrame([])

finaltrain = finaltrain._append({
                                        'Accuracy' : round(accuracy_score(y_true, y_pred)*100,3),
                                        'PrecisionTrain':round(precision_score(y_true, y_pred, average = 'weighted')*100,3),
                                        'RecallTrain':round(recall_score(y_true, y_pred, average = 'weighted')*100,3)  ,
                                        'F1Train':round(f1_score(y_true, y_pred, average = 'weighted')*100,3)}
                                      
                                        , ignore_index=True)
finaltrain 

### Extract features 

In [ ]:

model.eval()


features = {}

# Hook function to capture the output from any layer
def get_features(name):
    def hook(model, input, output):
        features[name] = output.detach()
    return hook

# Function to register the hook based on layer name
def register_layer_hook(model, layer_name):
    # Use eval() to access the layer dynamically
    layer = eval(f'model.{layer_name}')
    layer.register_forward_hook(get_features(layer_name))

# ===============================
# 3. Extract Features from ResNet's Specified Layer
# ===============================
# Assume train_loader and test_loader are defined DataLoader objects for your dataset

def extract_features(layer_name, train_loader, test_loader, model, device):
    # Register hook for the provided layer name
    register_layer_hook(model, layer_name)
    
    train_features_list = []  # To store features for training data
    train_labels_list = []  # To store labels for training data
    test_features_list = []  # To store features for test data
    test_labels_list = []  # To store labels for test data

    # Extract features from training dataset
    for images, labels in train_loader:
        with torch.no_grad():  # Disable gradient calculation
            images = images.to(device)
            
            # Forward pass through the model
            _ = model(images)
            
            # Extract layer features
            layer_features = features[layer_name]
            
            # Flatten features if needed (depends on the shape of your features)
            layer_features = layer_features.view(layer_features.size(0), -1)
            
            train_features_list.append(layer_features)
            train_labels_list.append(labels)

    # Extract features from test dataset
    for images, labels in test_loader:
        with torch.no_grad():
            images = images.to(device)
            
            # Forward pass through the model
            _ = model(images)
            
            # Extract layer features
            layer_features = features[layer_name]
            
            # Flatten features if needed
            layer_features = layer_features.view(layer_features.size(0), -1)
            
            test_features_list.append(layer_features)
            test_labels_list.append(labels)

    # Concatenate features and labels from the list to create tensors
    train_features = torch.cat(train_features_list)
    train_labels = torch.cat(train_labels_list)
    test_features = torch.cat(test_features_list)
    test_labels = torch.cat(test_labels_list)

    # ===============================
    # 4. Save Features to Files
    # ===============================
    # Create directory to store features if it doesn't exist
    output_dir = f'features/{layer_name}/'
    os.makedirs(output_dir, exist_ok=True)

    # Save train and test features and labels
    torch.save(train_features, os.path.join(output_dir, 'train_features.pt'))
    torch.save(train_labels, os.path.join(output_dir, 'train_labels.pt'))
    torch.save(test_features, os.path.join(output_dir, 'test_features.pt'))
    torch.save(test_labels, os.path.join(output_dir, 'test_labels.pt'))

    # Optional: save as .npy files if needed
    train_features_np = train_features.cpu().numpy()
    train_labels_np = train_labels.cpu().numpy()
    test_features_np = test_features.cpu().numpy()
    test_labels_np = test_labels.cpu().numpy()

    # Save as .npy files
    np.save(os.path.join(output_dir, 'train_features.npy'), train_features_np)
    np.save(os.path.join(output_dir, 'train_labels.npy'), train_labels_np)
    np.save(os.path.join(output_dir, 'test_features.npy'), test_features_np)
    np.save(os.path.join(output_dir, 'test_labels.npy'), test_labels_np)

    print(f"Features from '{layer_name}' saved in '{output_dir}'")

    return train_features, train_labels, test_features, test_labels

In [ ]:
layer_name = "Mixed_7c"  # Specify a layer from Inception-v3, e.g., Mixed_7c

# Call the function to extract features
train_features, train_labels, test_features, test_labels = extract_features(
    layer_name, train_loader, test_loader, model, device
)